In [12]:
# Google Gemini API client
from google import genai

# Standard libraries
import json          # Handle JSON data
import base64        # Encode/decode image data
import io            # Work with in-memory byte streams
import ast           # Safely evaluate Python literals
import re            # Clean and process text
import os            # File and directory operations

# Image processing
from PIL import Image

# UI framework
import gradio as gr

In [13]:
# Initialize Gemini client using API key from environment variables (Google AI Studio)

client = genai.Client(api_key=os.getenv('api_key'))

In [14]:
# Model Connectivity Test (Sample Prompt)

# Send a test prompt to verify the Gemini model is working correctly

response = client.models.generate_content(
    model = 'gemini-2.5-flash',
    contents = 'Provide Python code to reverse a string.'
)

print(response.text)

Python offers several straightforward ways to reverse a string. The most Pythonic and commonly used method is string slicing.

Here are a few methods, from the most Pythonic to more explicit loop-based approaches:

---

### Method 1: Using Slicing `[::-1]` (Most Pythonic and Recommended)

This is the simplest and most idiomatic way to reverse a string in Python. It creates a reversed copy of the string.

```python
def reverse_string_slice(s):
  """Reverses a string using slicing."""
  return s[::-1]

# --- Examples ---
print(f"Slice Method:")
print(f"'hello' reversed is: {reverse_string_slice('hello')}")         # Output: olleh
print(f"'Python' reversed is: {reverse_string_slice('Python')}")       # Output: nohtyP
print(f"'' reversed is: {reverse_string_slice('')}")                   # Output:
print(f"'a' reversed is: {reverse_string_slice('a')}")                 # Output: a
```

**Explanation:**
*   `s` is your string.
*   `[::-1]` creates a reversed copy. The first two `:` indicate a

In [15]:
# Recipe Generation Prompt Template

RECIPE_PROMPT_TEMPLATE = """
You are an Expert chef tasked with reducing food waste, specifically focused on home cooking and meal planning. Your goal is generate an ideal recipe based on available ingredients and a specific cuisine style.
Examine the following user inventory and cuisine preference and determine the best possible recipe that utilizes the inventory efficiently.

Here are the specific rules to follow when generating the recipe:
1. You must prioritize the ingridients provided in the inventory.
2. You may assume the user has basic pantry staples (salt,pepper,oil,water,basic spices).
3. The recipe must strictly adhere to the requested cuisine type (eg. if indian, use indian spice/methods).
4. Provide clear, step-by-step cooking instructions.

Focus on etracting the following details:
1. Recipe Name
2. List of ingredients (with quantities)
3.Step-by-step Instructions
4. Cooking time

Prioritise creating a cohesive, tasty dish over using every single random ingredient.
do NOT alter, rephrase the user's inventory items into things they didn't list (unless they are basic staples).

Exclude:
1. ingredients that are rare or highly specific if they are not in the inventory.
2. Extremely complex prefessional cooking techniques (keep it home-cook friendly)

Output your response as a single line JSON string according to the following structure and nothing else:
[
  {{
    'RecipeName':'Name of the dish',
    'Ingredients':['List','of','Ingredients','and quantities'],
    'Instructions':['Step 1...','Step 2...'],
    'PrepTime':'XX mins',
    'CuisineType':'The Cuisine Selected'
    }}
]

[INPUT DATA LABEL]:
Inventory:{inventory}
Cuisine Preference: {cuisine}
"""

In [16]:
# Model Input Declaration

# Ingredients available with the user
my_inventory = 'Paneer,onions,tomatoes,yogurt,garlic,ginger'

# Preferred cuisine type
desired_cuisine = 'Indian'

In [17]:
# Final Prompt Construction

# Populate the prompt template with user-provided inputs
final_prompt = RECIPE_PROMPT_TEMPLATE.format(
    inventory = my_inventory,
    cuisine = desired_cuisine
)

In [18]:
# List Available Gemini Models

# Retrieve all available models linked to the API key
models = client.models.list()

# Display model details
for model in models:
    print(model)

name='models/embedding-gecko-001' display_name='Embedding Gecko' description='Obtain a distributed representation of a text.' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo() input_token_limit=1024 output_token_limit=1 supported_actions=['embedText', 'countTextTokens'] default_checkpoint_id=None checkpoints=None temperature=None max_temperature=None top_p=None top_k=None thinking=None
name='models/gemini-2.5-flash' display_name='Gemini 2.5 Flash' description='Stable version of Gemini 2.5 Flash, our mid-size multimodal model that supports up to 1 million tokens, released in June of 2025.' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo() input_token_limit=1048576 output_token_limit=65536 supported_actions=['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent'] default_checkpoint_id=None checkpoints=None temperature=1.0 max_temperature=2.0 top_p=0.95 top_k=64 thinking=True
name='models/gemini-2.5-pro' display_na

In [19]:
# Generate Recipe Response from Gemini

# Generate a response using the Gemini text model and user inputs
response = client.models.generate_content(
    model = 'gemma-3-1b-it',
    contents = final_prompt
)

# Display the generated recipe
print(response.text)

```json
[
  {
    'RecipeName': 'Spicy Paneer & Tomato Curry with Basmati Rice',
    'Ingredients': ['Paneer', 'Onions', 'Tomatoes', 'Garlic', 'Ginger', 'Yogurt', 'Spices (Turmeric, Cumin, Coriander, Chili Powder, Garam Masala)', 'Basmati Rice', 'Salt', 'Oil'],
    'Instructions': [
      "1. Dice the onions, garlic, and ginger into small pieces.",
      "2. Heat oil in a pot over medium heat. Add the onions and sauté until translucent (about 5-7 minutes).",
      "3. Add the garlic and ginger and sauté for another minute until fragrant.",
      "4. Add the tomatoes and cook for 5-7 minutes, stirring occasionally, until they soften.",
      "5. Gently add the paneer and cook for 3-5 minutes, allowing it to brown slightly.",
      "6. Stir in the spices – turmeric, cumin, coriander, chili powder, and garam masala – and cook for 1 minute, allowing the spices to bloom.",
      "7. Pour in the yogurt and bring to a simmer.  Add salt to taste.",
      "8. Simmer for 10-15 minutes, allowing 

In [20]:
# Model Response Cleaning & Parsing

def clean_text(response):
    raw_text = response.text

     # Validate response content
    if not raw_text or not raw_text.strip():
        raise ValueError('Empty response from model')
    
    # Normalize and clean text
    text = raw_text.strip()
    text = re.sub(r"'''(?:json)?","",text).strip()

    # Extract JSON-like list from the response
    match = re.search(r"(\[.*\])",text,re.DOTALL)

    if not match:
        raise ValueError('No recipe data found')
    
    # Safely parse the extracted data
    data = ast.literal_eval(match.group(1))
    recipe_data = data[0]

    return (recipe_data['RecipeName'],
            recipe_data['Ingredients'],
            recipe_data['Instructions'],
            recipe_data['PrepTime'],
            recipe_data['CuisineType']
    )

clean_text(response)

('Spicy Paneer & Tomato Curry with Basmati Rice',
 ['Paneer',
  'Onions',
  'Tomatoes',
  'Garlic',
  'Ginger',
  'Yogurt',
  'Spices (Turmeric, Cumin, Coriander, Chili Powder, Garam Masala)',
  'Basmati Rice',
  'Salt',
  'Oil'],
 ['1. Dice the onions, garlic, and ginger into small pieces.',
  '2. Heat oil in a pot over medium heat. Add the onions and sauté until translucent (about 5-7 minutes).',
  '3. Add the garlic and ginger and sauté for another minute until fragrant.',
  '4. Add the tomatoes and cook for 5-7 minutes, stirring occasionally, until they soften.',
  '5. Gently add the paneer and cook for 3-5 minutes, allowing it to brown slightly.',
  '6. Stir in the spices – turmeric, cumin, coriander, chili powder, and garam masala – and cook for 1 minute, allowing the spices to bloom.',
  '7. Pour in the yogurt and bring to a simmer.  Add salt to taste.',
  '8. Simmer for 10-15 minutes, allowing the flavors to meld.',
  '9. Serve hot with basmati rice.'],
 20,
 'Indian')

In [21]:
# Final Recipe Output Presentation

def final_answer(response):
    recipeName, ingredients, instructions, prepTime, cuisineType = clean_text(response)

    print(f' RECIPE FOUND: {recipeName}')
    print(f' Time: {prepTime}')
    print(f' Cuisine: {cuisineType}')
    print('_'* 40)
    print(' INGREDIENTS:')
    for item in ingredients:
        print(f'- {item}')
        
    print('-' * 40)
    print(' INSTRUCTIONS:')
    for step in instructions:
        print(f'{step}')   

final_answer(response)

 RECIPE FOUND: Spicy Paneer & Tomato Curry with Basmati Rice
 Time: 20
 Cuisine: Indian
________________________________________
 INGREDIENTS:
- Paneer
- Onions
- Tomatoes
- Garlic
- Ginger
- Yogurt
- Spices (Turmeric, Cumin, Coriander, Chili Powder, Garam Masala)
- Basmati Rice
- Salt
- Oil
----------------------------------------
 INSTRUCTIONS:
1. Dice the onions, garlic, and ginger into small pieces.
2. Heat oil in a pot over medium heat. Add the onions and sauté until translucent (about 5-7 minutes).
3. Add the garlic and ginger and sauté for another minute until fragrant.
4. Add the tomatoes and cook for 5-7 minutes, stirring occasionally, until they soften.
5. Gently add the paneer and cook for 3-5 minutes, allowing it to brown slightly.
6. Stir in the spices – turmeric, cumin, coriander, chili powder, and garam masala – and cook for 1 minute, allowing the spices to bloom.
7. Pour in the yogurt and bring to a simmer.  Add salt to taste.
8. Simmer for 10-15 minutes, allowing the f

In [22]:
final_answer(response)

 RECIPE FOUND: Spicy Paneer & Tomato Curry with Basmati Rice
 Time: 20
 Cuisine: Indian
________________________________________
 INGREDIENTS:
- Paneer
- Onions
- Tomatoes
- Garlic
- Ginger
- Yogurt
- Spices (Turmeric, Cumin, Coriander, Chili Powder, Garam Masala)
- Basmati Rice
- Salt
- Oil
----------------------------------------
 INSTRUCTIONS:
1. Dice the onions, garlic, and ginger into small pieces.
2. Heat oil in a pot over medium heat. Add the onions and sauté until translucent (about 5-7 minutes).
3. Add the garlic and ginger and sauté for another minute until fragrant.
4. Add the tomatoes and cook for 5-7 minutes, stirring occasionally, until they soften.
5. Gently add the paneer and cook for 3-5 minutes, allowing it to brown slightly.
6. Stir in the spices – turmeric, cumin, coriander, chili powder, and garam masala – and cook for 1 minute, allowing the spices to bloom.
7. Pour in the yogurt and bring to a simmer.  Add salt to taste.
8. Simmer for 10-15 minutes, allowing the f

In [25]:
# Environment Configuration

# Password stored securely in environment variables
Password = os.getenv('PASSWORD')

# Authentication Logic
def verify_answer(pswd):
    if pswd == Password:
        return (
            'Access Granted !',
            gr.update(visible=True),
            gr.update(visible=True),
            gr.update(visible=True),
            gr.update(visible=True),
            gr.update(visible=False,value=''),
        )
    else:
        return(
            'Invalid credentials ! Please login with correct credentials',
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=False,value=''),
        )

# Prompt Template
RECIPE_PROMPT_TEMPLATE = """
You are an Expert chef tasked with reducing food waste, specifically focused on home cooking and meal planning. Your goal is generate an ideal recipe based on available ingredients and a specific cuisine style.
Examine the following user inventory and cuisine preference and determine the best possible recipe that utilizes the inventory efficiently.

Here are the specific rules to follow when generating the recipe:
1. You must prioritize the ingridients provided in the inventory.
2. You may assume the user has basic pantry staples (salt,pepper,oil,water,basic spices).
3. The recipe must strictly adhere to the requested cuisine type (eg. if indian, use indian spice/methods).
4. Provide clear, step-by-step cooking instructions.

Focus on etracting the following details:
1. Recipe Name
2. List of ingredients (with quantities)
3.Step-by-step Instructions
4. Cooking time

Prioritise creating a cohesive, tasty dish over using every single random ingredient.
do NOT alter, rephrase the user's inventory items into things they didn't list (unless they are basic staples).

Exclude:
1. ingredients that are rare or highly specific if they are not in the inventory.
2. Extremely complex prefessional cooking techniques (keep it home-cook friendly)

Output your response as a single line JSON string according to the following structure and nothing else:
[
  {{
    'RecipeName':'Name of the dish',
    'Ingredients':['List','of','Ingredients','and quantities'],
    'Instructions':['Step 1...','Step 2...'],
    'PrepTime':'XX mins',
    'CuisineType':'The Cuisine Selected'
    }}
]

[INPUT DATA LABEL]:
Inventory:{inventory}
Cuisine Preference: {cuisine}
"""

# Model Response Cleaning & Parsing
def clean_text(response):
    raw_text = response.text

    if not raw_text or not raw_text.strip():
        raise ValueError('Empty response from model')
    
    text = raw_text.strip()
    
    # Remove markdown formatting if present
    if "```" in text:
        text = text.replace("```json","").replace("```","").strip()

    # Extract JSON array
    start = text.find("[")
    end = text.rfind("]")

    if start == -1 or end == -1:
        raise ValueError("JSON array not found in model output")
    
    json_text = text[start:end+1]
    json_text= json_text.replace("'",'"')

    try:
        data = json.loads(json_text)
    except json.JSONDecodeError:
        data = ast.literal_eval(json_text)

    recipe_data = data[0]
 
    return (recipe_data["RecipeName"],
            recipe_data["Ingredients"],
            recipe_data["Instructions"],
            recipe_data["PrepTime"],
            recipe_data["CuisineType"]
    )

# Model Execution Logic
def model_run(Ingredients,desired_cuisine):

    if not Ingredients or not desired_cuisine:
        return gr.update(value='Please enter both ingredients and cuisine.',visible=True)
    
    final_prompt = RECIPE_PROMPT_TEMPLATE.format(
    inventory = Ingredients,
    cuisine = desired_cuisine
    )

    try:
        response = client.models.generate_content(
            model = 'gemma-3-1b-it',
            contents = final_prompt
        )
    except Exception as e:
        return gr.update(value=f'Model Error: {str(e)}', visible=True) 
        
    try:
        name,ing,steps,time,ctype = clean_text(response)
    except Exception as e:
        return gr.update(value=f"Parsing error: {str(e)}\n\nRaw response: {response.text[:500]}",visible=True)
    
    output = f"""
RECIPE: {name}
Time: {time}
Cuisine: {ctype}

    Ingredients:
    """ + '\n'.join(f'- {i}' for i in ing) + "\n\nINSTRUCTIONS:\n" + "\n".join(f' {i+1}. {step}' for i,step in enumerate(steps))

    return gr.update(value=output.strip(),visible=True)

# Clear Inputs
def clear_all():
    return (
        gr.update(value='',visible=True),
        gr.update(value='',visible=True),
        gr.update(value='',visible=False)
    )

# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
        <div align="center">
          <h1 style="font-size: 42px; color: #2c3e50;">
                    Gemini Recipe Generator
          </h1>
        </div>
        """)
    
    password = gr.Textbox(label = 'Password',type='password',placeholder='Enter password to access')
    auth_message = gr.Textbox(label="Status", interactive=False)
    Ingredients = gr.Textbox(label = 'Enter Ingredients (Seperated by Coma)',visible=False,placeholder='eg....Paneer,onions,tomatoes,yogurt,garlic,ginger')
    desired_cuisine = gr.Textbox(label = 'Enter Cuisine Style',visible=False,placeholder='eg...Indian,Italian,Chinese,Mexican')
    submit = gr.Button('Generate Recipe',visible=False,variant='primary')
    clear = gr.Button('New Recipe',visible=False)
    Response = gr.Textbox(label = 'Your Recipe is Ready',visible=False)

    password.submit(verify_answer,
                    inputs=password,
                    outputs=[auth_message,Ingredients,desired_cuisine,submit,clear,Response])

    submit.click(model_run,
                inputs=[Ingredients,desired_cuisine],
                outputs=Response)
    
    clear.click(
        clear_all,
        inputs=None,
        outputs=[Ingredients,desired_cuisine,Response]
    )
demo.launch(share=True)

C:\Users\youar\AppData\Local\Temp\ipykernel_36724\3874923906.py:147: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://e5978b933a07f3f04f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
